# Компьютерная семантика

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/howto/wordnet.html
* https://ruwordnet.ru/ru
* https://habr.com/ru/companies/unistar_digital/articles/687148/
* https://www.scaler.com/topics/nlp/wordnet-in-nlp/

## Задачи для совместного разбора

1\. Дано описание классов `Sence` и `Synset`. Создайте объект `Synset` на основе файлов из `data/rwn`.

In [1]:
from dataclasses import dataclass, field

@dataclass
class Sense:
    id: str
    name: str
    lemma: str
    main_word: str
    synt_type: str
    poses: str
    synset_id: str
    meaning: str

@dataclass
class Synset:
    id: str
    ruthes_name: str
    definition: str
    part_of_speech: str
    senses: list[Sense] = field(default_factory=list)
    hyponyms: list["Synset"] = field(default_factory=list, repr=False)
    hypernyms: list["Synset"] = field(default_factory=list, repr=False)

In [2]:
from bs4 import BeautifulSoup

In [3]:
with open("synsets.N.xml", encoding='utf8') as fp:
  synsets_bs = BeautifulSoup(fp)

In [4]:
example_synset = synsets_bs.find("synset")
example_synset

<synset definition="" id="N12658" part_of_speech="N" ruthes_name="КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ">
<sense id="115643">МЕДИЦИНСКИЙ КОДИРОВАНИЕ</sense>
<sense id="115640">КОДИРОВАНИЕ</sense>
<sense id="115641">КОДИРОВАНИЕ ОТ ЗАВИСИМОСТЬ</sense>
<sense id="115642">КОДИРОВАНИЕ ЗАВИСИМОСТЬ</sense>
</synset>

In [5]:
example_synset.attrs

{'id': 'N12658',
 'ruthes_name': 'КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ',
 'definition': '',
 'part_of_speech': 'N'}

In [6]:
synset = Synset(**example_synset.attrs)

In [ ]:
synset.id

'N12658'

In [ ]:
with open("senses.N.xml", "r", encoding="utf8") as fp:
  senses_bs = BeautifulSoup(fp)

In [ ]:
senses_bs.find('sense')

<sense id="115643" lemma="МЕДИЦИНСКИЙ КОДИРОВАНИЕ" main_word="КОДИРОВАНИЕ" meaning="1" name="МЕДИЦИНСКОЕ КОДИРОВАНИЕ" poses="Adj N" synset_id="N12658" synt_type="NG"></sense>

In [ ]:
senses_list = [
    Sense(**sense.attrs)
    for sense in senses_bs.find_all('sense', synset_id=synset.id)
]

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. На основе файлов `rwn/synsets.A.xml`, `rwn/synsets.N.xml` и `rwn/synsets.V.xml` создайте словарь `synsets`, где ключом является ID синсета, а значением - объекта класса `Synset`. Поля `senses`, `hyponyms`, `hypernyms` оставьте со значениями по умолчанию. Выведите количество объектов в полученном словаре на экран.

In [7]:
files = ['synsets.A.xml', 'synsets.N.xml', 'synsets.V.xml']

In [8]:
synsets = {}
for file_ in files:
  with open(file_, encoding='utf8') as fp:
    synsets_bs = BeautifulSoup(fp)
  for s in synsets_bs.find_all("synset"):
    synset = Synset(**s.attrs)
    synsets[s.attrs['id']] = synset

In [9]:
import itertools
dict(itertools.islice(synsets.items(), 2))

{'A1': Synset(id='A1', ruthes_name='ПОДПИСКА НА ПЕРИОДИЧЕСКИЕ ИЗДАНИЯ', definition='', part_of_speech='Adj', senses=[]),
 'A2': Synset(id='A2', ruthes_name='РАВНОМЕРНЫЙ', definition='', part_of_speech='Adj', senses=[])}

In [ ]:
len(synsets)

49492

<p class="task" id="2"></p>

2\. Обновите поле `senses` у объектов в словаре `synsets` на основе файлов `rwn/senses.A.xml`, `rwn/senses.N.xml` и `rwn/senses.V.xml`. Выведите на экран среднее количество синонимов у синсетов.

In [10]:
files = ['senses.A.xml', 'senses.N.xml', 'senses.V.xml']
for file_ in files:
  with open(file_, encoding='utf8') as fp:
    senses_bs = BeautifulSoup(fp)
  for s in senses_bs.find_all("sense"):
    synsets[s['synset_id']].senses.append(s)

In [11]:
import itertools
dict(itertools.islice(synsets.items(), 2))

{'A1': Synset(id='A1', ruthes_name='ПОДПИСКА НА ПЕРИОДИЧЕСКИЕ ИЗДАНИЯ', definition='', part_of_speech='Adj', senses=[<sense id="95459" lemma="ПОДПИСНОЙ" main_word="" meaning="2" name="ПОДПИСНОЙ" poses="" synset_id="A1" synt_type="Adj"></sense>]),
 'A2': Synset(id='A2', ruthes_name='РАВНОМЕРНЫЙ', definition='', part_of_speech='Adj', senses=[<sense id="122916" lemma="РАВНОМЕРНЫЙ" main_word="" meaning="1" name="РАВНОМЕРНЫЙ" poses="" synset_id="A2" synt_type="Adj"></sense>])}

In [ ]:
mean = 0
for v in synsets.values():
  mean += len(v.senses)
mean / len(synsets)

2.6350925402085186

<p class="task" id="3"></p>

3\. Проанализируйте, какие типы отношений представлены в файле `rwn/relation.xml`. Выведите множество типов отношений на экран.
Обновите поля `hyponyms` и `hypernyms` у объектов в словаре `synsets` на основе файла `rwn/relation.xml`.

In [12]:
files = ['synset_relations.A.xml', 'synset_relations.N.xml', "synset_relations.V.xml"]

In [13]:
relation_types = set()
for file_ in files:
  with open(file_, encoding='utf8') as fp:
    relations_bs = BeautifulSoup(fp)
  for r in relations_bs.find_all('relation'):
    relation_types.add(r['name'])
    if r['name'] == "hypernym":
      synsets[r['parent_id']].hypernyms.append(r)
    if r['name'] == "hyponym":
      synsets[r['parent_id']].hyponyms.append(r)

In [ ]:
relation_types

{'POS-synonymy',
 'antonym',
 'cause',
 'domain',
 'entailment',
 'hypernym',
 'hyponym',
 'instance hypernym',
 'instance hyponym',
 'part holonym',
 'part meronym'}

In [ ]:
synsets['A2'].hypernyms[0]

<relation child_id="A4670" name="hypernym" parent_id="A2"></relation>

In [ ]:
synsets['A2'].hyponyms

[<relation child_id="A8709" name="hyponym" parent_id="A2"></relation>,
 <relation child_id="A11873" name="hyponym" parent_id="A2"></relation>]

<p class="task" id="4"></p>

4\. Напишите функцию `find_by_name`, которая ищет синсеты по вхождению заданного слова в поле `ruthes_name`. При поиске приводите введенное слово к нормальной форме и не учитывайте регистр слов. Функция должна вернуть список, отсортированный по возрастаю значений расстояния Левенштейна между названием синсета и введенным словом. Продемонстрируйте, какие синсеты находятся по слову "собака".

In [14]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b660eea3aaa5d6abaf1588b777dce76fef8537b812cb0be68c8fd6ab43354860
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [15]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [16]:
from nltk.metrics.distance import edit_distance

In [17]:
def find_by_name(name):
  res = []
  name_ = morph.parse(name)[0].normal_form.lower()
  for synset in synsets.values():
    ruthes_name = morph.parse(synset.ruthes_name)[0].normal_form.lower()
    if name in ruthes_name:
      res.append(synset)
  res = sorted(res, key=lambda x: edit_distance(name_, x.ruthes_name))
  return res

In [ ]:
find_by_name('собака')

[Synset(id='A6565', ruthes_name='СОБАКА', definition='хищное млекопитающее семейства псовых', part_of_speech='Adj', senses=[<sense id="4612" lemma="ПСИНЫЙ" main_word="" meaning="1" name="ПСИНЫЙ" poses="" synset_id="A6565" synt_type="Adj"></sense>, <sense id="4946" lemma="ПЕСИЙ" main_word="" meaning="1" name="ПЕСИЙ" poses="" synset_id="A6565" synt_type="Adj"></sense>, <sense id="5115" lemma="СОБАЧИЙ" main_word="" meaning="1" name="СОБАЧИЙ" poses="" synset_id="A6565" synt_type="Adj"></sense>]),
 Synset(id='N18450', ruthes_name='СОБАКА', definition='хищное млекопитающее семейства псовых', part_of_speech='N', senses=[<sense id="5046" lemma="ПЕСИК" main_word="" meaning="1" name="ПЕСИК" poses="" synset_id="N18450" synt_type="N"></sense>, <sense id="5113" lemma="ПЕС" main_word="" meaning="2" name="ПЕС" poses="" synset_id="N18450" synt_type="N"></sense>, <sense id="5114" lemma="СОБАКА" main_word="" meaning="1" name="СОБАКА" poses="" synset_id="N18450" synt_type="N"></sense>, <sense id="5116" l

<p class="task" id="5"></p>

5\. Для пары слов "собака" и "кошка" найдите ближайший общий родительский синсет и выведите на экран его название. Синсет `A` назовем родительским синсетом синсета `B`, если от `B` можно подняться к `A` в таксономии синсетов, используя отношения гиперонимии. Найдите общий родительский синсет для пары слов "кошка" и "студент" и выведите на экран его название.

In [38]:
cats = list(
    map(
      lambda x: x.hypernyms,
      find_by_name('кошка')
    )
)
dogs = list(
    map(
      lambda x: x.hypernyms,
      find_by_name('собака')
    )
)

In [39]:
cats = [item for sublist in cats for item in sublist]
dogs = [item for sublist in dogs for item in sublist]

In [40]:
hyps = []
for cat in cats:
  for dog in dogs:
    if dog['child_id'] == cat['child_id']:
      hyps.append(dog['child_id'])

In [ ]:
synsets[hyps[0]].ruthes_name

'ЖИВОТНОЕ'

In [41]:
students = list(
    map(
      lambda x: x.hypernyms,
      find_by_name('студент')
    )
)

In [42]:
students = [item for sublist in students for item in sublist]

In [43]:
hyps = []
while cats != []:
  for cat in cats:
    for student in students:
      if student['child_id'] == cat['child_id']:
        hyps.append(student['child_id'])
  cats = list(map(lambda x: synsets[x['child_id']].hypernyms, cats))
  students = list(map(lambda x: synsets[x['child_id']].hypernyms, students))
  cats = [item for sublist in cats for item in sublist]
  students = [item for sublist in students for item in sublist]

In [47]:
synsets[hyps[0]].ruthes_name

'ЖИВОЙ ОРГАНИЗМ'

<p class="task" id="6"></p>

6\. Для каждого слова из представленного текста найдите все возможные синонимы. Набор синонимов получите на основе поля `senses` у объектов `Synset`. В случае обнаружения точного совпадения введенного слова хотя бы с одним элементом из `senses`, весь набор `senses` трактуйте как синонимы. При поиске приводите слово к нормальной форме и не учитывайте регистр слов.

Составьте все возможные вариации данного предложения, заменяя слова на найденные синонимы. В случае отсутствия синонимов для какого-то из слов, используйте исходное слово для генерации предложения. Выведите на экран общее количество перефразированных предложений.

In [ ]:
text = "Студент ужаснулся когда увидел задачу"

In [ ]:
def find_synonyms(word):
  synonyms = []
  word = morph.parse(word)[0].normal_form.lower()
  found = find_by_name(word)
  for synset in found:
    for sense in synset.senses:
      if word in sense['name'].lower().split():
        return [s['name'].lower() for s in synset.senses]
  return []

In [ ]:
synonyms = []
for w in text.split():
  if len(find_synonyms(w)) != 0:
    synonyms.append(find_synonyms(w))
  else:
    synonyms.append([w])

In [ ]:
from itertools import product

In [ ]:
for element in itertools.product(*synonyms):
    print(' '.join(element))

студент стынет кровь когда иметь встречу задача
студент стынет кровь когда видеть задача
студент стынет кровь когда увидеть задача
студент стынет кровь когда видаться задача
студент стынет кровь когда видеться задача
студент стынет кровь когда повидать задача
студент стынет кровь когда встретить задача
студент стынет кровь когда встречать задача
студент стынет кровь когда свидеться задача
студент стынет кровь когда увидеться задача
студент стынет кровь когда пересечься задача
студент стынет кровь когда встретиться задача
студент стынет кровь когда встречаться задача
студент стынет кровь когда пересекаться задача
студент стынет кровь когда повидаться задача
студент стынет кровь в жилах когда иметь встречу задача
студент стынет кровь в жилах когда видеть задача
студент стынет кровь в жилах когда увидеть задача
студент стынет кровь в жилах когда видаться задача
студент стынет кровь в жилах когда видеться задача
студент стынет кровь в жилах когда повидать задача
студент стынет кровь в жила

## Обратная связь
- [x] Хочу получить обратную связь по решению